In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Matrix Product State (MPS)

The goal of this problem is to convert a general N-site wavefunction=state into MPS forms.
Consider a general N-site state
$$
  |\psi\rangle = \sum_{s_1, s_2, \cdots, s_N} \psi_{s_1, s_2, \cdots, s_N} |s_1, s_2, \cdots, s_N\rangle,
$$
where $s_i=1,2$.
We want to convert the coefficient $\psi_{s_1, s_2, \cdots, s_N}$ into one of the following MPS form. Here we use the case of $N=4$ as an example
* $A_1 A_2 A_3 S_s B_4$
* $A_1 S_1 B_2 B_3 B_4$

More specifically,
* $\psi_{s_1, s_2, \cdots, s_N} = [A_1(s_1)]_{1,2} [A_2(s_2)]_{2,4} [A_3(s_3)]_{4,2} [S_3]_{2,2} [B_4(s_4)]_{2,1}$.
* $\psi_{s_1, s_2, \cdots, s_N} = [A_1(s_1)]_{1,2} [S_1]_{2,2} [B_2(s_2)]_{2,4} [B_3(s_3)]_{4,2}  [B_4(s_4)]_{2,1}$.

Here $[S_i]$ are diagonal matrix. $[A_i(s_i)]$ satisfies left-canonical condition and $[B_i(s_i)]$ satisfies right-canonical condition.
We can further decompose above MPS into the Gamma-Lambda form:
* $GA_1 S_1 GA_2 S_2 GB_3 S_3 GB_4$

where $GA_i(s_i) S_i=A_i(s_i)$ and $B_j(s_j)=S_j G_B(s_j)$.


# Problem-2

For a fixed N. (you can start with N=4 and see how far you can go.)
* Generate a random state psi and normalize it.
* Decompose psi into AA...ASB form and ASB...B form. (change A to B at first or last site.)
* Check that you can obtain the original coefficient from the matrix product of A, S, B.
* Decompose psi into Gamma-Lambda form. (change A to B at middle.)
* Can you obtain obtain representation that chanes A to B any any position?

In [3]:
psi = np.arange(2**4).reshape([2, 2, 2, 2])
#print(psi)
norm = np.linalg.norm(psi.flatten())
psi = psi/norm
np.linalg.norm(psi.flatten())
print(psi.shape)
print(psi)
norm = np.linalg.norm(psi.flatten())
print(norm)

(2, 2, 2, 2)
[[[[0.         0.02839809]
   [0.05679618 0.08519428]]

  [[0.11359237 0.14199046]
   [0.17038855 0.19878664]]]


 [[[0.22718473 0.25558283]
   [0.28398092 0.31237901]]

  [[0.3407771  0.36917519]
   [0.39757328 0.42597138]]]]
0.9999999999999999


# Singular values at each position

In [4]:
N = 4
for i in range(1, N):
    psi_i = psi.reshape(2**i, 2**(N-i))
    print("shape=", psi_i.shape)
    s = np.linalg.svd(psi_i, compute_uv=False)
    print("singular values=", s)
    print("norm=", np.sum(s**2))

shape= (2, 8)
singular values= [0.99288132 0.11910791]
norm= 1.0000000000000002
shape= (4, 4)
singular values= [9.97907911e-01 6.46513855e-02 2.41763294e-17 5.66154335e-19]
norm= 1.0000000000000002
shape= (8, 2)
singular values= [0.99956248 0.02957795]
norm= 1.0000000000000002


# Decomposition into MPS

In [13]:
psi = psi.reshape(2, 8)
print("cut = 1 | 2, 3, 4")
print("shape to be SVD")
print(psi.shape)

A1, s1, vh1 = np.linalg.svd(psi, full_matrices=False)
print("shape after SVD")
print(A1.shape,s1.shape, vh1.shape)
print("s1")
print(s1)
print("A1[0], A1[1]")
print(A1[0,:])
print(A1[1,:])

cut = 1 | 2, 3, 4
shape to be SVD
(2, 8)
shape after SVD
(2, 2) (2,) (2, 8)
s1
[0.99288132 0.11910791]
A1[0], A1[1]
[-0.31874637 -0.94784005]
[-0.94784005  0.31874637]


In [16]:
print("merge s1, vh1 into R")
R = np.diag(s1) @ vh1
print(R.shape)

print("cut = 1, 2 | 3, 4")
R = R.reshape(2*2, 2*2)
print("shape to be SVD")
print(R.shape)

A2, s2, vh2 = np.linalg.svd(R, full_matrices=False)
print("shape after SVD")
print(A2.shape, s2.shape, vh2.shape)
print("s2")
print(s2)

print("reshape A2")
A2 = A2.reshape(2, 2, 4)
print(A2.shape, s2.shape, vh2.shape)
print("A2[0], A2[1]")
print(A2[:, 0, :])
print(A2[:, 1, :])

merge s1, vh1 into R
(2, 8)
cut = 1, 2 | 3, 4
shape to be SVD
(4, 4)
shape after SVD
(4, 4) (4,) (4, 4)
s2
[9.97907911e-01 6.46513855e-02 9.65225281e-17 1.26896884e-17]
reshape A2
(2, 2, 4) (4,) (4, 4)
A2[0], A2[1]
[[-0.54529035  0.32205939  0.7613669  -0.13876825]
 [ 0.08647788  0.76906291 -0.35852688 -0.52203668]]
[[-0.83190012 -0.16651309 -0.52898285 -0.01981744]
 [-0.05587671  0.5263966  -0.1093438   0.84132535]]


In [18]:
print("merge s2, vh2 into R")
print(s2)
print(vh2)
R = np.diag(s2) @ vh2
print(R.shape)

print("cut = 1, 2 , 3 | 4")
R = R.reshape(2*2*2, 2)
print("shape to be SVD")
print(R.shape)

A3, s3, B4 = np.linalg.svd(R, full_matrices=False)
print("shape after SVD")
print(A3.shape, s3.shape, B4.shape)
print("s3")
print(s3)

print("reshape A3")
A3 = A3.reshape(4, 2, 2)
print(A3.shape, s3.shape, B4.shape)
print("A3[0], A3[1]")
print(A3[:, 0, :])
print(A3[:, 1, :])

print("B4[0], B4[1]")
print(B4)
print(B4[:, 0])
print(B4[:, 1])

merge s2, vh2 into R
[9.97907911e-01 6.46513855e-02 9.65225281e-17 1.26896884e-17]
[[ 0.42334086  0.47243254  0.52152422  0.57061589]
 [ 0.72165263  0.27714165 -0.16736932 -0.6118803 ]
 [-0.48564538  0.45874662  0.53944288 -0.51254413]
 [-0.25327568  0.6996796  -0.63953216  0.19312824]]
(4, 4)
cut = 1, 2 , 3 | 4
shape to be SVD
(8, 2)
shape after SVD
(8, 2) (2,) (2, 2)
s3
[0.99956248 0.02957795]
reshape A3
(4, 2, 2) (2,) (2, 2)
A3[0], A3[1]
[[-6.33295716e-01  1.32708441e-01]
 [-4.46398854e-02 -7.61138809e-01]
 [-1.29485292e-18  2.17964905e-15]
 [-4.41774414e-18  2.82182300e-16]]
[[-7.71754775e-01 -9.46323875e-02]
 [ 3.65839276e-02 -6.27774557e-01]
 [ 1.65138450e-18 -2.42763429e-15]
 [ 3.63985784e-18  2.58875848e-16]]
B4[0], B4[1]
[[-0.6719584  -0.74058889]
 [-0.74058889  0.6719584 ]]
[-0.6719584  -0.74058889]
[-0.74058889  0.6719584 ]


# original coefficient from the matrix product of A, S, B

In [24]:
c = A1 @ np.diag(s1) @ vh1
print(c)
print(np.trace(c))
a = np.diag(s1) @ vh1
b = A1 @ a
print(b)
print(np.trace(b))

[[5.55111512e-17 2.83980917e-02 5.67961834e-02 8.51942751e-02
  1.13592367e-01 1.41990459e-01 1.70388550e-01 1.98786642e-01]
 [2.27184734e-01 2.55582825e-01 2.83980917e-01 3.12379009e-01
  3.40777101e-01 3.69175192e-01 3.97573284e-01 4.25971376e-01]]
0.25558282541117916
[[4.16333634e-17 2.83980917e-02 5.67961834e-02 8.51942751e-02
  1.13592367e-01 1.41990459e-01 1.70388550e-01 1.98786642e-01]
 [2.27184734e-01 2.55582825e-01 2.83980917e-01 3.12379009e-01
  3.40777101e-01 3.69175192e-01 3.97573284e-01 4.25971376e-01]]
0.2555828254111792
